In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('E:\\Downloads\\train2_irf_0000_corr98.csv', dtype='int8')
y = pd.read_csv('E:\\Downloads\\y2.csv', sep=';', usecols=[1], dtype='int8').values.ravel()

print(df.shape, y.shape)

(26495, 8065) (26495,)


In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, GRU, Bidirectional
from keras.optimizers import RMSprop
import h5py

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=.3, stratify=y)

forward_lstm = LSTM(input_dim=1024, output_dim=512, return_sequences=True)
backward_lstm = LSTM(input_dim=1024, output_dim=512, return_sequences=True)
#brnn = Bidirectional(forward=forward_lstm, backward=backward_lstm, return_sequences=True)

av_roc = 0.
model = Sequential()
conv_layer = Convolution1D(input_dim=4,
                        input_length=1000,
                        nb_filter=1024,
                        filter_length=30,
                        border_mode="valid",
                        activation="relu",
                        subsample_length=1)

callbacks = [EarlyStopping(monitor='val_loss', patience=100, verbose=1)]
#checkpointer = ModelCheckpoint(filepath="DanQ-JASPAR_bestmodel.hdf5", verbose=1, save_best_only=True)

#conv_layer.set_weights(conv_weights)
model.add(conv_layer)
model.add(MaxPooling1D(pool_length=15, stride=15))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(input_dim=1024, output_dim=512, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(input_dim=64*1024, output_dim=925, activation="relu"))
model.add(Dense(input_dim=925, output_dim=919, activation="sigmoid"))

print ('compiling model')
model.compile(loss='binary_crossentropy', optimizer='rmsprop', class_mode="binary")
model.fit(X_train, y_train, batch_size=100, nb_epoch=32, shuffle=True, validation_data=(X_test, y_test), callbacks=callbacks)

valid_preds = model.predict_proba(X_test, verbose=0)
valid_preds = valid_preds[:, 1]
roc = roc_auc_score(y_test, valid_preds)
print("ROC:", roc)
av_roc += roc

#print('Average ROC:', av_roc/nb_folds)

C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:15: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:15: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 102..., units=512)`
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:16: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:16: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(return_sequences=True, input_shape=(None, 102..., units=512)`
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:27: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(1000, 4)

compiling model


C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=65536, units=925)`
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", input_dim=925, units=919)`
C:\Users\Paul\Anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


ValueError: Error when checking input: expected conv1d_4_input to have 3 dimensions, but got array with shape (18546, 8065)

In [46]:
    preds = model.predict_proba(X_test, verbose=0)[:, 1]
    submission = pd.DataFrame(preds, index=ids, columns=['target'])
    submission.to_csv('Keras_BTB.csv')    

NameError: name 'model' is not defined